# Basics of Keras in Tensorflow

In [1]:
import numpy as np
from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler

* The data is custom made for a hypotethical clinical new drug test in 2100 individuals which consists a group of people who are younger than 65 years of age and another group with equal number of people who are 65 years of age or older.
* Around 95% of patients who are 65 and older exprienced side effects.
* Around 95% of pateints who are under 65 years of age experienced no side effects.


In [2]:
X = [] # input data
Y = [] # label

In [3]:
# Creating the outliers
total_patients = 2100
no_of_outliers = int(5 / 100 * total_patients / 2)
for i in range(no_of_outliers):
    # ~5% of young patients who experienced side effects.
    random_young_age_patient = randint(13, 64)
    X.append(random_young_age_patient)
    Y.append(1)
    
    # ~5% of old patients who did not experience side effects
    random_old_age_patient = randint(65, 100)
    X.append(random_old_age_patient)
    Y.append(1)

In [4]:
# Rest of patients apart from outliers
no_of_non_outliers = 2100 - no_of_outliers
for i in range(no_of_non_outliers):
    # ~95% of young patients who did not experience side effetcs.
    random_young_age_patient = randint(13, 64)
    X.append(random_young_age_patient)
    Y.append(0)
    
    # ~95% of old patients who experienced side effects.
    random_old_age_patient = randint(65, 100)
    X.append(random_old_age_patient)
    Y.append(1)

In [5]:
# Converting the lists to np arrays.
X = np.array(X)
Y = np.array(Y)
X, Y = shuffle(X, Y)

In [6]:
scaler = MinMaxScaler(feature_range = (0, 1))
# data should not be 1D for fit_transfor
scaled_X = scaler.fit_transform(X.reshape(-1, 1))

### Simple tf.keras Sequnetial Model

In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

In [16]:
physical_devices = tf.config.experimental.list_logical_devices('GPU')
print("No Of GPUs = ", len(physical_devices))
if len(physical_devices) != 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

No Of GPUs =  0


In [20]:
# Creating Fully Connected Layers. 
model = Sequential([
    Dense(units = 16, input_shape = (1, ), activation = 'relu'),
    Dense(units = 32, activation = 'relu'),
    Dense(units = 2, activation = 'softmax')
])

In [22]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 16)                32        
_________________________________________________________________
dense_7 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 66        
Total params: 642
Trainable params: 642
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.compile(optimizer = Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics= ['accuracy'])

In [25]:
model.fit(x=X, y=Y, batch_size=10, epochs=30, verbose=2)

Epoch 1/30
420/420 - 0s - loss: 0.1209 - accuracy: 0.9619
Epoch 2/30
420/420 - 0s - loss: 0.1086 - accuracy: 0.9700
Epoch 3/30
420/420 - 0s - loss: 0.1087 - accuracy: 0.9698
Epoch 4/30
420/420 - 0s - loss: 0.1074 - accuracy: 0.9683
Epoch 5/30
420/420 - 0s - loss: 0.1024 - accuracy: 0.9714
Epoch 6/30
420/420 - 0s - loss: 0.1045 - accuracy: 0.9681
Epoch 7/30
420/420 - 0s - loss: 0.1038 - accuracy: 0.9683
Epoch 8/30
420/420 - 0s - loss: 0.1011 - accuracy: 0.9705
Epoch 9/30
420/420 - 0s - loss: 0.0995 - accuracy: 0.9710
Epoch 10/30
420/420 - 0s - loss: 0.1003 - accuracy: 0.9714
Epoch 11/30
420/420 - 0s - loss: 0.0974 - accuracy: 0.9729
Epoch 12/30
420/420 - 0s - loss: 0.1035 - accuracy: 0.9667
Epoch 13/30
420/420 - 0s - loss: 0.0989 - accuracy: 0.9695
Epoch 14/30
420/420 - 0s - loss: 0.0991 - accuracy: 0.9700
Epoch 15/30
420/420 - 0s - loss: 0.0998 - accuracy: 0.9681
Epoch 16/30
420/420 - 0s - loss: 0.1007 - accuracy: 0.9674
Epoch 17/30
420/420 - 0s - loss: 0.0907 - accuracy: 0.9755
Epoch 